In [2]:
import spacy
from spacy import displacy
from spacy.util import filter_spans
import re
import pandas as pd
import pickle
nlp = spacy.load('en_core_web_sm')
#snt = nlp.create_pipe('sentencizer')

from spacy.matcher import Matcher 
from spacy.tokens import Span 
import networkx as nx
from networkx.drawing.nx_agraph import graphviz_layout
import matplotlib.pyplot as plt
from tqdm import tqdm
from spacy.lang.en import English
%matplotlib inline

In [46]:
string = """Brown bears have long claws. The Brown bear has a brown fur. 
The Brown bear has a black nose.
he has a black nose.
The bear has a purple belly.
Their feet are orange.
The brown bear is similar to the black bear.
"""


In [57]:
doc = nlp(sentences_cleaned[0])

In [76]:
doc[-1].morph

PunctType=Peri

In [59]:
dir(doc)

['_',
 '__bytes__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__ne__',
 '__new__',
 '__pyx_vtable__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__unicode__',
 '_bulk_merge',
 '_get_array_attrs',
 '_py_tokens',
 '_realloc',
 '_vector',
 '_vector_norm',
 'cats',
 'char_span',
 'copy',
 'count_by',
 'doc',
 'ents',
 'extend_tensor',
 'from_array',
 'from_bytes',
 'from_dict',
 'from_disk',
 'from_docs',
 'get_extension',
 'get_lca_matrix',
 'has_annotation',
 'has_extension',
 'has_unknown_spaces',
 'has_vector',
 'is_nered',
 'is_parsed',
 'is_sentenced',
 'is_tagged',
 'lang',
 'lang_',
 'mem',
 'noun_chunks',
 'noun_chunks_iterator',
 'remove_extension',
 'retokenize',
 'sentiment',
 'sents',
 'set_ents',
 'se

In [58]:
[print(i) for i in doc.noun_chunks]

brown bears
very large and curved claws
the forelimbs
the hind limbs


[None, None, None, None]

In [53]:
string = """
Brown bears have very large and curved claws, those present on the forelimbs being longer than those on the hind limbs. They may reach 5 to 6 centimetres (2.0 to 2.4 in) and may measure 7 to 10 centimetres (2.8 to 3.9 in) along the curve.[69] They are generally dark with a light tip, with some forms having completely light claws.[62] Brown bear claws are longer and straighter than those of American black bears (Ursus americanus).[61] The claws are blunt, while those of a black bear are sharp. Due to their claw structure, in addition to their excessive weight, adult brown bears cannot typically climb trees as well as both species of black bear, although in rare cases adult female brown bears have been seen in trees.[70] The claws of a polar bear are also quite different, being notably shorter but broader with a strong curve and sharper point, presumably both as an aid to traveling over ice (sometimes nearly vertically) and procuring active prey.[23][71] The paws of the brown bear are quite large. The rear feet of adult bears have been found to typically measure 21 to 36 cm (8.3 to 14.2 in) long, while the forefeet tend to measure about 40% less in length. All four feet in average sized brown bears tend to be about 17.5 to 20 cm (6.9 to 7.9 in) in width. In large coastal or Kodiak bear males, the hindfoot may measure up to 40 cm (16 in) in length, 28.5 cm (11.2 in) in width, while outsized Kodiak bears having had confirmed measurements of up to 46 cm (18 in) along their rear foot.[72][73][74] Brown bears are the only extant bears with a hump at the top of their shoulder, which is made entirely of muscle, this feature having developed presumably for imparting more force in digging, which is habitual during foraging for most bears of the species and also used heavily in den construction prior to hibernation.[61][75] The brown bear's strength has been roughly estimated as 2.5 to 5 times that of a human.[76]
"""

In [54]:
def TextCleaner(text, splitComma=False, splitPointComma=False):
    
    """
    Split text into sentences and cleans them 
    from e.g. whitespaces and references.
    """
    
    # Split on commas
    if splitComma:
        text = text.replace(', ', '. ')
    if splitPointComma:
        text = text.replace('; ', '. ')
    text = nlp(text)
    # Init result
    result = []
    # Init regexes
    TextCleaner = [
    ' \(\d+.+?Close\n\t\n\)',
    '\[\d*\]',
    '\([^)]*\)'
    ]
    # Extract single senteces if large enough
    sentences = [str(sent) for sent in text.sents if len(sent) >= 3]
        
    for sentence in sentences:
        #print(sentence.lower())
        # Apply the regexes
        for Cleaner in TextCleaner:
            sentence = re.sub(Cleaner, '', sentence, flags=re.DOTALL)
        # Clean tabs and newlines
        sentence = sentence.replace('\n', '')
        sentence = sentence.replace('\t', '')
        # Add to result
        result.append(sentence.lower())
    
    return result

def RelationExtractor(sentences):
    
    """
    Extract Relations from sentences that are in list.
    """
    
    # Init result
    result = []
    # Init matcher
    matcher = Matcher(nlp.vocab)
    # Relation patterns
    RelationPatterns = [
                [{"DEP": "ROOT"}, {"DEP": "acomp", "OP": "?"}, {"DEP": "prep", "OP": "?"}],
                [{"DEP": "auxpass"}], 
    ]
    # Add the matcher
    matcher.add("Description", RelationPatterns)
    
    # Loop over sentences
    for sentence in sentences:
        # nlp sentence
        sentence = nlp(sentence)
        # Relations
        relations = []
        # Apply matcher     
        matches = matcher(sentence)
        if matches:
            for _, start, end in matches:
                # The matched span 
                extracted = sentence[start:end]
                relations.append(extracted)
            #print(relations)
            # Extract the span with most information            
            filteredRelation = filter_spans(relations)
            # Append result
            result.append(str(filteredRelation[0].lemma_).lower())
        else:
            result.append(None)
    
    return result

def SubjectExtractor(sentences):
    
    """
    Extract Subjects from sentences that are in list.
    """
    
    # Init result
    result = []
    # Init matcher
    matcher = Matcher(nlp.vocab)
    # Relation patterns
    SubjectsPattern = [
                [{"DEP": "advmod", "OP": "?"}, {"DEP": "amod"}, {"DEP": "nsubj"}],
                [{"DEP": "compound", "OP": "?"}, {"DEP": "nsubj"}],
                [{"DEP": "amod", "OP": "?"}, {"DEP": "nsubjpass"}]
    ]
    # Add the matcher
    matcher.add("Description", SubjectsPattern)
    # Subject memory
    temp = ''
    
    # Loop over sentences
    for sentence in sentences:
        # nlp sentence
        sentence = nlp(sentence)
        # Relations
        subjects = []
        # Apply matcher     
        matches = matcher(sentence)
        if matches:
            for _, start, end in matches:
                # The matched span 
                extracted = sentence[start:end]
                subjects.append(extracted)
            # Extract the span with most information            
            filteredSubjects = filter_spans(subjects)
            # Append result
            result.append(str(filteredSubjects[0].lemma_).lower())
        else:
            result.append(None)
    
    return result

def ObjectExtractor(sentences):
    
    """
    Extract Objects from sentences that are in list.
    """
    
    # Init result
    result = []
    # Init matcher
    matcher = Matcher(nlp.vocab)
    # Relation patterns
    ObjectsPattern = [
                [{"DEP": "nsubj"}, {"DEP": "advcl"}, {"DEP": "acomp"}],
                [{"DEP": "advmod"}, {"DEP": "acomp", "OP": "?"}, {"DEP": "cc", "OP": "?"}, {"DEP": "conj", "OP": "?"}],
                [{"DEP": "advmod"}, {"DEP": "neg", "OP": "?"}, {"DEP": "advmod", "OP": "?"}, {"DEP": "ROOT", "OP": "?"}],
                [{"DEP": "amod"}, {"DEP": "probj"}],
                [{"DEP": "amod", "OP": "?"}, {"DEP": "npadvmod", "OP": "?"}, {"DEP": "punct", "OP": "?"}, {"DEP": "amod", "OP": "?"}, {"DEP": "dobj"}],
                [{"DEP": "quantmod", "OP": "?"},{"DEP": "quantmod", "OP": "?"},{"DEP": "nummod", "OP": "?"},{"DEP": "amod", "OP": "?"}, {"DEP": "pobj"}],

    ]
    # Add the matcher
    matcher.add("Description", ObjectsPattern)
    
    # Loop over sentences
    for sentence in sentences:
        # nlp sentence
        sentence = nlp(sentence)
        # Relations
        objects = []
        # Apply matcher     
        matches = matcher(sentence)
        if matches:
            for _, start, end in matches:
                # The matched span 
                extracted = sentence[start:end]
                objects.append(extracted)
            # Extract the span with most information            
            filteredObjects = filter_spans(objects)
            # Append result
            result.append(str(filteredObjects[0].lemma_).lower())
        else:
            result.append(None)
    
    return result

In [55]:
sentences_cleaned = TextCleaner(string)
sentences_cleaned

['brown bears have very large and curved claws, those present on the forelimbs being longer than those on the hind limbs.',
 'they may reach 5 to 6 centimetres  and may measure 7 to 10 centimetres  along the curve.',
 'they are generally dark with a light tip, with some forms having completely light claws.',
 'brown bear claws are longer and straighter than those of american black bears .',
 'the claws are blunt, while those of a black bear are sharp.',
 'due to their claw structure, in addition to their excessive weight, adult brown bears cannot typically climb trees as well as both species of black bear, although in rare cases adult female brown bears have been seen in trees.',
 'the claws of a polar bear are also quite different, being notably shorter but broader with a strong curve and sharper point, presumably both as an aid to traveling over ice  and procuring active prey.',
 'the paws of the brown bear are quite large.',
 'the rear feet of adult bears have been found to typicall

In [44]:
S, R, O = tuple([SubjectExtractor(sentences_cleaned), RelationExtractor(sentences_cleaned), ObjectExtractor(sentences_cleaned)])

for s,r,o in zip(S,R,O):
    print(s, '--', r, '--', o)

brown bear -- have -- very
they -- reach -- centimetre
they -- be -- generally dark
bear claw -- be long -- those
claw -- be blunt -- black bear
brown bear -- climb -- structure
claws -- be -- polar bear
paw -- be -- brown bear
rear foot -- be -- bear
foot -- tend -- brown bear
hindfoot -- measure -- large coastal
strength -- be -- roughly estimate


In [45]:
displacy.render(nlp(sentences_cleaned[0]), style="dep")

In [ ]:
for Token in nlp(sentences_cleaned[0]):
    print(Token.dep_, '-', Token.pos_, '-', Token.tag_ ,'>', Token.text)

In [ ]:
def ExtractFeatures(sentence):
    
    """
    Extract features from text.
    """
    
    
    
    # Init result
    result = None
    # Init subject
    subject = ''
    # Init matcher
    matcher = Matcher(nlp.vocab)
    # Init empty list
    features = []
    # Init patterns
    DescriptionPatterns = [
                [{"DEP": "amod", "OP": "?"}, {"DEP": "cc", "OP": "?"}, {"DEP": "conj", "OP": "?"}, {"DEP": "dobj"}],
                [{"DEP": "nsubj"}, {"DEP": "ROOT"}, {"DEP": "acomp"}, {"DEP": "cc", "OP": "?"}, {"DEP": "conj", "OP": "?"}],
    ]
    
    # Add the matcher
    matcher.add("Description", DescriptionPatterns)
    # Apply matcher     
    matches = matcher(sentence)
    #print(matches)
    
    if matches:
        #print(matches)
        for _, start, end in matches:
            # The matched span 
            extracted = sentence[start:end]
            # Append all the matches 
            features.append(extracted)
    # Extract the span with most information            
    filteredFeatures = filter_spans(features)
    #print(filteredFeatures)
    
    for feature in filteredFeatures:
        #print(feature)
        
        # Drop empty features
        if str(feature[0:-1]) == '':
            continue

        # Return the correct string
        if feature[-1].pos_ == "NOUN":
            result = 'The ' + str(feature[-1].lemma_) + ' is ' + str(' '.join([i.lemma_ for i in feature[0:-1]]))
            
        else:
            result = 'The ' + str(feature[0].lemma_) + ' is ' + str(' '.join([i.lemma_ for i in feature[2:]]))
            
    # Try to find a result        
    if result == None:
        for token in sentence:
            #print(token.dep_)
            if token.dep_ == "nsubj" and token.pos_ != "PRON":
                subject = token
                
                DescriptionPatterns = [
                    [{"DEP": "advmod", "OP": "?"}, {"DEP": "acomp"}],
                ]

                # Add the matcher
                matcher.add("Description", DescriptionPatterns)
                # Apply matcher     
                matches = matcher(sentence)
                                
                if matches:
                    #print(matches)
                    for _, start, end in matches:
                        # The matched span 
                        extracted = sentence[start:end]
                        #print(extracted)
                        # Append all the matches 
                        features.append(extracted)

                # Extract the span with most information            
                filteredFeatures = filter_spans(features)
                
                for feature in filteredFeatures:
                    #print(feature)
                
                    result = 'The ' + str(subject.lemma_) + ' is ' + str(' '.join([i.lemma_ for i in feature]))
           
    
    return result

In [ ]:
key = 'Brown bear'
ExtractFeatures(sentences[8], key)

In [ ]:
features = [ExtractFeatures(sent) for sent in tqdm(sentences)]
features

In [ ]:
# extract subject
source = [i[0] for i in Knowledge]
# extract relations
relations = [i[1] for i in Knowledge]
# extract object
target = [i[2] for i in Knowledge]

In [ ]:
kg_df = pd.DataFrame({'source':source, 'edge':relations, 'target':target,})

In [ ]:
kg_df

In [ ]:
# create a directed-graph from a dataframe
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

In [ ]:
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
nx.draw_networkx_edge_labels(G, pos=pos)
plt.show()

### Clean Part

In [ ]:
pickle_in = open("../data/processed/dataWiki_animals.pkl", "rb")
#pickle_in = open("../data/processed/dataBOW.pkl", "rb")

data = pickle.load(pickle_in)

In [ ]:
data = data['Brown bear']
data = list(set(data))

In [ ]:
data = [i[1] for i in data if i[0] == 1]

In [ ]:
data[0]

In [ ]:
doc = nlp(data[0])

In [ ]:
Sentences = [sent for sent in doc.sents if len(sent) >= 3]

In [ ]:
for tok in Sentences[0]:
    print(tok.dep_)

In [ ]:
def SubjectRelationObject(tokens):
    
    """
    Returns the subject, the relation and the object of 
    a single sentence.
    """
    
    #tokens = nlp(Sentence)
    
    # Init empty variables
    Previous = Next_ = None
    Future = len(tokens)
    Subject = ''
    Object = ''
    Relation = ''
    SubjectConstruction = ''
    ObjectConstruction = ''
    Memory_I = ''
    UsedPrevious = False
    
    # Init Dep lists
    RelationList = ["ROOT", "adj", "attr", "agent",]
    SubjectObjectList = ["compound", "prep", "conj", "mod", "amod", "acomp"]
    PossesiveList = ["poss"]
    MemoryObjectorSubjectList = ["amod", "compound", "acomp"]
    
    # Loop over the tokens in the sentence
    for Index, Token in enumerate(tokens):

        # Check memory
        if any(subs in Token.dep_ for subs in MemoryObjectorSubjectList):
            Memory_I = Token.text.strip()
            #print('YESSS')

        # Skip if the token is a point/comma etc.
        if "punct" in Token.dep_:
            continue
            
        ### Relation Part    
        # Check if token exists in Relation List    
        if any(subs in Token.dep_ for subs in RelationList):
            # Update variable with token.lemma
            Relation = Token.lemma_
        
        ### Subject/Object Part                
        if "subj" in Token.dep_:
            if Memory_I:
                Subject = Memory_I + ' ' + Token.lemma_
            else:
                Subject = Token.lemma_
        
        if "obj" in Token.dep_:
            if Object:
                Relation = Relation + ' ' + Object
                Object = Memory_I + ' ' + Token.lemma_
            if Memory_I:
                Object = Memory_I + ' ' + Token.lemma_
            else:
                Object = Token.lemma_
                
    return [Subject.strip().lower(), Relation.strip().lower(), Object.strip().lower()]

In [ ]:
SubjectRelationObject(sentences[7])